In [2]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter
from matplotlib.colors import LightSource, LinearSegmentedColormap
import matplotlib.cm as cm
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning, module="pyvista.jupyter")
warnings.filterwarnings("ignore", message=".*tight_layout.*")

# Configuration parameters
SMOOTH_SIGMA = 3.0
DETAIL_OCTAVES = 4
DETAIL_PERSISTENCE = 0.5
DETAIL_SCALE = 10.0
ELEV = 30
AZIM = -60
INVERT = True
SCALE_TO = (200, 0)
INPUT_DIR = 'gen_images'
OUTPUT_DIR = 'Inference_3d Plots'

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Custom terrain colormap
def create_terrain_cmap():
    colors = [
        (0.0, (0.0, 0.0, 0.5)),  # Deep blue (ocean)
        (0.1, (0.0, 0.4, 0.8)),  # Medium blue
        (0.2, (0.2, 0.6, 0.9)),  # Light blue
        (0.3, (0.8, 0.8, 0.2)),  # Sand/beach
        (0.4, (0.1, 0.6, 0.1)),  # Lowland green
        (0.6, (0.4, 0.8, 0.4)),  # Highland green
        (0.7, (0.6, 0.5, 0.3)),  # Low mountains
        (0.8, (0.5, 0.4, 0.3)),  # High mountains
        (0.9, (0.9, 0.9, 0.9)),  # Lower snow
        (1.0, (1.0, 1.0, 1.0))   # Upper snow
    ]
    cdict = {'red': [], 'green': [], 'blue': []}
    for pos, color in colors:
        cdict['red'].append((pos, color[0], color[0]))
        cdict['green'].append((pos, color[1], color[1]))
        cdict['blue'].append((pos, color[2], color[2]))
    return LinearSegmentedColormap('terrain_natural', cdict)

terrain_cmap = create_terrain_cmap()

# Load height map from image
def load_height_map(path, scale_to=None, invert=True):
    img = Image.open(path).convert("L")
    arr = np.array(img, dtype=np.float32) / 255.0
    if invert:
        arr = 1.0 - arr
    if scale_to is not None:
        hi, lo = scale_to
        arr = arr * (hi - lo) + lo
    return arr

# Smooth terrain
def smooth_terrain(height_map, sigma=1.0):
    return gaussian_filter(height_map, sigma=sigma)

# Add fractal detail with fallback
def add_opensimplex_detail(height_map, octaves=3, persistence=0.5, scale=10.0):
    try:
        from opensimplex import OpenSimplex
        h, w = height_map.shape
        detail = np.zeros((h, w))
        noise_gen = OpenSimplex(seed=np.random.randint(0, 1000000))
        for y in range(h):
            for x in range(w):
                value = 0
                amplitude = 1.0
                frequency = 1.0
                for _ in range(octaves):
                    nx = x / w * frequency * 5
                    ny = y / h * frequency * 5
                    value += noise_gen.noise2(nx, ny) * amplitude
                    amplitude *= persistence
                    frequency *= 2
                detail[y, x] = value
        detail = (detail - detail.min()) / (detail.max() - detail.min()) * scale
        return height_map + detail
    except ImportError:
        print("OpenSimplex not installed. Using NumPy fallback.")
        return add_numpy_fractal_detail(height_map, octaves, persistence, scale)

def add_numpy_fractal_detail(height_map, octaves=3, persistence=0.5, scale=10.0):
    h, w = height_map.shape
    detail = np.zeros((h, w))
    for octave in range(octaves):
        freq = 2**octave
        amp = persistence**octave
        octave_size = (max(h // freq, 1), max(w // freq, 1))
        noise = np.random.rand(*octave_size)
        from scipy.ndimage import zoom
        zoomed = zoom(noise, (h/noise.shape[0], w/noise.shape[1]), order=1)
        detail += zoomed * amp
    detail = (detail - detail.min()) / (detail.max() - detail.min()) * scale
    return height_map + detail

# Enhance terrain
def enhance_terrain(height_map, smooth_sigma=1.0, add_detail=True, 
                   detail_octaves=3, detail_persistence=0.5, detail_scale=10.0):
    terrain = smooth_terrain(height_map, sigma=smooth_sigma)
    if add_detail:
        terrain = add_opensimplex_detail(terrain, octaves=detail_octaves,
                                         persistence=detail_persistence, scale=detail_scale)
    terrain = np.clip(terrain, 0, 200)
    if terrain.max() > terrain.min():
        terrain = (terrain - terrain.min()) / (terrain.max() - terrain.min()) * 200
    else:
        terrain = np.zeros_like(terrain)
    return terrain

# Plot 3D terrain with Matplotlib
def plot_matplotlib_3d(z, title="3D Terrain", cmap=terrain_cmap, elev=30, azim=-60, 
                       with_hillshade=True, output_prefix=""):
    filename = os.path.join(OUTPUT_DIR, f"{output_prefix}_3D_Terrain.png")
    cmap_obj = cmap if isinstance(cmap, LinearSegmentedColormap) else cm.get_cmap(cmap)
    h, w = z.shape
    x = np.arange(w)
    y = np.arange(h)
    x, y = np.meshgrid(x, y)
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection="3d")
    norm_z = (z - z.min()) / (z.max() - z.min() if z.max() > z.min() else 1)
    if with_hillshade:
        ls = LightSource(azdeg=315, altdeg=45)
        rgb = ls.shade(z, cmap=cmap_obj, vert_exag=0.3, blend_mode='soft')
        surf = ax.plot_surface(x, y, z, facecolors=rgb, linewidth=0, antialiased=True, 
                              rcount=200, ccount=200)
    else:
        surf = ax.plot_surface(x, y, z, cmap=cmap_obj, linewidth=0, antialiased=True, 
                              rcount=200, ccount=200, vmin=0, vmax=200)
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label="Elevation")
    ax.set_title(title, fontsize=16)
    ax.set_xlabel("X", fontsize=12)
    ax.set_ylabel("Y", fontsize=12)
    ax.set_zlabel("Elevation", fontsize=12)
    ax.set_zlim(0, 200)
    ax.view_init(elev=elev, azim=azim)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved: {filename}")

# Main function to process images
def process_images(max_images):
    # Validate input directory
    if not os.path.exists(INPUT_DIR):
        print(f"Input directory {INPUT_DIR} does not exist.")
        return
    
    # Get list of image files
    valid_extensions = ('.png', '.jpg', '.jpeg', '.tif', '.tiff')
    image_files = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith(valid_extensions)]
    
    if not image_files:
        print(f"No valid images found in {INPUT_DIR}.")
        return
    
    # Limit to max_images
    image_files = sorted(image_files)[:max_images]
    print(f"Processing {len(image_files)} images from {INPUT_DIR}...")
    
    # Process each image
    for image_name in image_files:
        image_path = os.path.join(INPUT_DIR, image_name)
        base_name = os.path.splitext(image_name)[0]
        output_name = f"{base_name}_3D_Terrain.png"
        output_path = os.path.join(OUTPUT_DIR, output_name)
        
        try:
            # Load and enhance the image
            z_original = load_height_map(image_path, scale_to=SCALE_TO, invert=INVERT)
            z_enhanced = enhance_terrain(z_original, smooth_sigma=SMOOTH_SIGMA, add_detail=True,
                                         detail_octaves=DETAIL_OCTAVES, detail_persistence=DETAIL_PERSISTENCE,
                                         detail_scale=DETAIL_SCALE)
            
            # Generate and save 3D terrain plot
            plot_matplotlib_3d(z_enhanced, title=f"Enhanced 3D Terrain for {base_name}", 
                               cmap=terrain_cmap, elev=ELEV, azim=AZIM, with_hillshade=True, 
                               output_prefix=base_name)
            
            print(f"Processed: {output_name}")
        except Exception as e:
            print(f"Error processing {image_name}: {e}")
            continue

# User input for number of images to process
try:
    max_images = int(input("Enter the number of images to process (e.g., 2 or 3): "))
    if max_images <= 0:
        print("Please enter a positive number.")
    else:
        process_images(max_images)
except ValueError:
    print("Invalid input. Please enter a valid number.")

Processing 4 images from gen_images...
Saved: Inference_3d Plots\0000_3D_Terrain.png
Processed: 0000_3D_Terrain.png
Saved: Inference_3d Plots\0001_3D_Terrain.png
Processed: 0001_3D_Terrain.png
Saved: Inference_3d Plots\0002_3D_Terrain.png
Processed: 0002_3D_Terrain.png
Saved: Inference_3d Plots\0003_3D_Terrain.png
Processed: 0003_3D_Terrain.png


2025-04-26 05:59:09.595 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 05:59:09.888 
  command:

    streamlit run C:\Users\TEJAS\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-26 05:59:09.889 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 05:59:09.890 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 05:59:09.891 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 05:59:09.892 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 05:59:09.892 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-26 05:59:09.892 Thre

<function __main__.cleanup()>